In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import Image

In [2]:
# Параметры изображения и эмоций
img_size = (48, 48)
emotions = ["Anger", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprised"]
amountofemotions = len(emotions)
data_papka = "archive/emotions"

In [3]:
# Функция для загрузки данных
def loading_info(data_papka, img_size, emotions):
    img = []
    label = []
    for i, j in enumerate(emotions):
        emotion_papka = os.path.join(data_papka, j)
        if os.path.isdir(emotion_papka):
            for img_file in os.listdir(emotion_papka):
                img_path = os.path.join(emotion_papka, img_file)
                try:
                    img_j = Image.open(img_path).convert("L")
                    img_j = img_j.resize(img_size)
                    img_array = np.array(img_j) / 255
                    img.append(img_array)
                    label.append(i)
                except Exception as e:
                    print("error", e)
    return np.array(img), np.array(label)

In [4]:
# Загрузка данных
X, y = loading_info(data_papka, img_size, emotions)

In [ ]:
# Преобразование данных
X = X.reshape(-1, img_size[0], img_size[1], 1)
y = to_categorical(y, num_classes=amountofemotions)

In [ ]:
# Разделение данных на обучающую, валидационную и тестовую выборки
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_value, X_test, y_value, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Создание модели
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(amountofemotions, activation='softmax')
])

In [ ]:
# Компиляция модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Приведение типов данных
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_value = X_value.astype("float32")
y_test = y_test.astype("float32")
y_train = y_train.astype("float32")
y_value = y_value.astype("float32")

In [ ]:
# Обучение модели
history = model.fit(X_train, y_train, validation_data=(X_value, y_value), epochs=50, batch_size=16)

In [ ]:
# Сохранение модели в формате .h5
model.save("model.h5")
print("Модель сохранена как model.h5")